<a href="https://colab.research.google.com/github/hrbolek/learning/blob/master/operanalyst/linearprog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Příklady

### Příklad 1

Surový materiál je dodáván v délce 7 m. Požadavek je produkovat délky 2.3 m (A), 1.9 m (B) a 0.8 m (C) v počtech A 60 ks, B 90 ks a C 320 ks. Pro jeden řez předpokládejme prořez 0.005 m. Stanovte minimální počet kusů surového materiálu.